Variational Autoencoder (VAE) es un modelo generativo.
- Esto significa que estamos intentando modelar como **generamos** los datos (es decir, aprender la distribución de los mismos), no solo como mapeamos $x \rightarrow y$, como haciamos en aprendizaje supervisado.
- Entonces, nos preguntamos: _¿Como pueden haberse generado estos datos $x$ en la vida real?_
- Respuesta: De una variable latente $z$ que captura la estructura no observada, y de un proceso condicional $x∼p_θ(x∣z)$. 

**Modelo de Variable Latente**

Forma probabilistica estándar de cualquier modelo probabilistico de variable latente:

$$ p_θ(z,x) = p_θ(z)p_θ(x|z)$$

Donde:
- $z$: variable latente (factores no observados).
    - Ejemplo: para una oración podría ser: tematica (deportes, politica, ...), sentimiento (positivo, neutral, negativo), estilo (formal, casual), etc

- $p_θ(z)$: prior sobre las variables latentes (en general, distribución normal estándar)
 
- $p_θ(x|z)$: decoder que mapea del espacio latente al espacio de datos

- $p_θ(z,x)$: probabilidad conjunta sobre las variables latentes y observadas.

¿Por qué empezamos modelando desde acá?
- VAE quiere modelar $p(x)$ (la distribución de los datos) usando variables latentes. Esto quiere decir:

$$ p_θ(x) = \int p_θ(z,x)dz = \int p_θ(z)p_θ(x|z)dz $$

- Es decir:
1. Primero generás un latente $z$ desde una distribución conocida, como $N(0,I)$
2. Luego generás un dato $x$ a partir de ese $z$.

👉 Pero en la realidad solo observás $x$, no $z$. Entonces, si querés saber qué tan probable es un dato $x$ según tu modelo, tenés que sumar (integrar) sobre todos los posibles valores que pudo haber tenido $z$.

- Pero... esa integral es intractable, por eso usamos una aproximación: el ELBO.


Entonces volvamos al inicio:

$$ p_θ(z,x) = p_θ(z)p_θ(x|z) $$

- Dijimos que $p_θ(z)$ suele ser una Gaussiana.
- $p_θ(x|z)$ (_“Dado un valor z en el espacio latente, ¿cómo se ve un dato x?”_) suele ser el producto de distribuciones de la familia exponencial.
    - _¿Porque producto de distribuciones?_ Porque normalmente suponemos que las componentes de x son independientes entre sí dado z. Entonces podemos escribir: $p_θ(x|z) = \prod_{i=1}^D p_θ(x_i|z)$. Simplifica el calculo
    - _¿Porque esas distribuciones?_
        - Si los datos son binarios (ej: pixel blanco o negro) usamos $x_i ∈ {0,1} ⇒ p_θ(x_i∣z)=Bernoulli(μ_i(z))$. Entonces: $p_θ(x|z) = \prod_{i=1}^D Bernoulli(x_i; \mu_i(z))$. Entonces la red nos da los $μ_i(z) ∈(0,1)$, o sea, las probabilidades de que cada píxel sea 1.
        
        - Si los datos son continuos (como imágenes en escala de grises normalizadas o audio): $x_i ∈ \R ⇒ p_θ(x_i∣z)=\mathcal{N}(x_i;μ_i(z), \sigma²)$. En general se asume varianza fija: $p_θ(x|z) = \prod_{i=1}^D \mathcal{N}(x_i;μ_i(z), \sigma²)$. Entonces la red nos devuelve los $μ_i(z)$ y opcionalmente los $\sigma_i²$

    - _¿Porque tiene sentido hacerlo asi?_
        - Desde el punto de vista probabilístico, necesitamos una función de densidad para evaluar la probabilidad del dato x dado un z.
        - Desde el punto de vista computacional, multiplicar distribuciones independientes simplifica el logaritmo.
        - Desde el punto de vista del aprendizaje, nos permite definir la función de pérdida (log-likelihood) y derivarla para actualizar los pesos de la red.